# Diamonds Data Analysis

In [ ]:
# Sadra
# testsdasd

In [ ]:
# Sadra 2

In [ ]:
# Joshua
# test test

In [ ]:
# Joshua 2
# test test

In [1]:
# Maiah
### Run this cell before continuing.
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

In [16]:
# Maiah 2

# Reading in the data set
url <- "https://raw.githubusercontent.com/sadrakhosravi/g1-dsci-project/main/diamonds.csv"

diamond_data <- read_csv("https://raw.githubusercontent.com/sadrakhosravi/g1-dsci-project/main/diamonds.csv")

# Tidy the data by making the cut a factor, changing color names to colorless and faint color, changing column names, and only selecting data needed.
diamond_data <- diamond_data |>
                 mutate(cut = as_factor(cut)) |>   # making cut a factor
                 mutate(color = fct_recode(color, "Colorless" = "D", "Colorless" = "E", "Colorless" = "F" , "Faint Color" = "G", "Faint Color" = "H",
                                           "Faint Color" = "I", "Faint Color" = "J")) |>  # changing names to correspond with color
                 rename(length = x, width = y, depth = z, total_depth_percentage = depth) |> # renaming column names
                 select(carat:color, total_depth_percentage:depth)  # taking out data not needed (clarity)
diamond_data

# Split diamond_data into training and test data
diamond_split <- initial_split( diamond_data, prop = 0.75, strata = cut)
diamond_train <- training(diamond_split)
diamond_test <- testing(diamond_split)

# Find proportions of each different type of cut based on training data. Also identifies if there are any NA's that need to be filtered out
diamond_proportions <- diamond_train |>
                      group_by(cut) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(diamond_train))

diamond_proportions



Rows: 53940 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): cut, color, clarity
dbl (7): carat, depth, table, price, x, y, z

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


carat,cut,color,total_depth_percentage,table,price,length,width,depth
<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.23,Ideal,Colorless,61.5,55,326,3.95,3.98,2.43
0.21,Premium,Colorless,59.8,61,326,3.89,3.84,2.31
0.23,Good,Colorless,56.9,65,327,4.05,4.07,2.31
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.70,Very Good,Colorless,62.8,60,2757,5.66,5.68,3.56
0.86,Premium,Faint Color,61.0,58,2757,6.15,6.12,3.74
0.75,Ideal,Colorless,62.2,55,2757,5.83,5.87,3.64


cut,n,percent
<fct>,<int>,<dbl>
Ideal,16125,39.861073
Premium,10327,25.528391
Good,3691,9.124169
Very Good,9080,22.445801
Fair,1230,3.040566


In [ ]:


# Dhruv

In [ ]:
# Dhruv 2